In [1]:
import re
import pandas as pd
from pathlib import Path

In [2]:
def load_df(path, flnames):
    df = []
    for flname in flnames:
        df_tmp = pd.read_csv(path.joinpath(flname), low_memory=False)
        
        # add year
        match = re.match(r'.*([1-3][0-9]{3})', flname)
        if match is not None:
            # Then it found a match!
            year = match.group(1)
            print(year)
        df_tmp['year_data'] = year

        df.append(df_tmp)
    df = pd.concat(df, axis = 0, ignore_index=True)
    return df

data_path = Path("G:\My Drive\Colab Notebooks\Zillow\data")
if (not 'dfall' in locals()) and data_path.joinpath('dfall.pkl').is_file():
    print('load cache')  
    dfall = pd.read_pickle(data_path.joinpath('dfall.pkl')) 
elif (not 'dfall' in locals()) or (not data_path.joinpath('dfall.pkl').is_file()):
    # mix type found without disable low memory mode
    # option: low_memory=False
    properties_flnames = ['properties_2016.csv', 'properties_2017.csv']
    train_flname = ['train_2016_v2.csv', 'train_2017.csv'] 
    df_prop = load_df(data_path, properties_flnames)
    df_trn = load_df(data_path, train_flname)
    dfall = df_prop.merge(df_trn, left_on = ['parcelid', 'year_data'], right_on=['parcelid', 'year_data'], how='outer')
    dfall.to_pickle(data_path.joinpath('dfall.pkl'))
    print('save cache')

display(dfall)

load cache


In [ ]:
dfall.info()

In [3]:
# fix mix type
# https://medium.com/@dataproducts/pandas-detect-mixed-data-types-and-fix-it-7f0bd3b8d071

for colname in dfall.columns:
    col_tmp = dfall[colname]
    col_tmp_nona = col_tmp[~col_tmp.isna()]
    print(colname)
    print(col_tmp_nona.unique())

parcelid
[ 10754147  10759547  10843547 ... 168040630 168040830 168040430]
airconditioningtypeid
[ 1. 13.  5. 11.  9. 12.  3.]
architecturalstyletypeid
[ 7. 21.  8.  2.  3.  5. 10. 27.]
basementsqft
[ 216.  555.  224.  782.  651.  516.  600.  732. 1145.  220.  420.  688.
  144.  540. 1312.  690.  640.  550.   63.  198. 1120.  924. 1572. 1035.
  195.  544.  498.  176.  360.  154.  240.  616. 1131.  200. 1016. 1089.
 2443.  276.  396.  149.  168.  495.  208.  560.  939.  704.  460. 1152.
  462. 1904.  888.  170.  625.  450.   72.  252.  637.   96. 1172. 1464.
  787.  501.  473.  318.  112.  394.  985.  169.  400.  826.  780. 1350.
  610. 1528.  255.  164.  315.  836.  630.  160. 1508.  775.  676.  448.
  756.  912.  179.  700.  348.  595.  231.  100.  266.  620.  757.  786.
  872.  672.  831. 1544.  946.  180.  313.  314.  440.  528.   36.  320.
  515.  196.  119. 3374.  606.  736.  678. 1282.  135.   64.  506.  414.
 1296. 1197. 1112.  814.  530.  165. 1314.  352.  282.  293.  805. 2025

In [4]:
if (not 'dfall_typemod' in locals()) and data_path.joinpath('dfall_typemod.pkl').is_file():
    print('load cache')
    dfall_typemod = pd.read_pickle(data_path.joinpath('dfall_typemod.pkl')) 
elif (not 'dfall_typemod' in locals()) or (not data_path.joinpath('dfall_typemod.pkl').is_file()):
    # covert year_data to string
    dfall['year_data'] = dfall['year_data'].astype('string')

    # fix object (column with mix types)
    # find column with mix type
    dfall_mixtypeidx = []
    dfall_mixtypecolname = []
    for idx, value in enumerate(dfall.dtypes.items()):
        if value[1] == "object":
            dfall_mixtypeidx.append(idx)
            dfall_mixtypecolname.append(value[1])
    print(f'column idx with mix type: {dfall_mixtypeidx}')

    dfall_mixtypecolname = dfall.columns[dfall_mixtypeidx]
    print(f'column with mix type: {dfall_mixtypecolname}')
    dfall_mixtype = dfall.iloc[:, dfall_mixtypeidx]
    # dfall_mixtype.info()

    # print data types without na
    for mix_idx in dfall_mixtypeidx:
        col = dfall.iloc[:, mix_idx]
        col_nona = col[~col.isna()]
        if isinstance(col_nona.unique()[0], bool):
            print('is bool')
            dfall[dfall.columns[mix_idx]] = dfall[dfall.columns[mix_idx]].fillna(False).astype('bool')
        elif col_nona.unique()[0] == 'Y':
            print('is Y')
            dfall[dfall.columns[mix_idx]] = dfall[dfall.columns[mix_idx]].fillna(False)
            dfall[dfall.columns[mix_idx]] = dfall[dfall.columns[mix_idx]].replace('Y', True)
            dfall[dfall.columns[mix_idx]] = dfall[dfall.columns[mix_idx]].astype('bool')
        elif isinstance(col_nona.unique()[0], str):
            print('is str')
            dfall[dfall.columns[mix_idx]] = dfall[dfall.columns[mix_idx]].fillna('').astype('string')
    dfall.iloc[:, dfall_mixtypeidx].info()
    
    # convert float to int if no decimal
    dfall_floatidx = []
    dfall_floatcolname = []
    for idx, value in enumerate(dfall.dtypes.items()):
        if value[1] == "float64":
            dfall_floatidx.append(idx)
            dfall_floatcolname.append(value[1])
    print(dfall_floatidx)

    def check_num_int(nums):
        status = True
        for num in nums:
            if num % 1 != 0:
                return False
        return status

    for colidx in dfall_floatidx:
        col_unique = pd.Series(dfall.iloc[:, colidx].unique()).dropna()
        #print(col_unique)

        allint_status = check_num_int(col_unique)
        if allint_status:
            dfall[dfall.columns[colidx]] = dfall[dfall.columns[colidx]].astype('Int64')


    # change year to string
    datecols = ['yearbuilt', 'assessmentyear', 'year_data']
    dfall[datecols] = dfall[datecols].astype('Int64') 
    
    # change id and code to string
    stringcol = ['parcelid', 
             'airconditioningtypeid', 
             'architecturalstyletypeid', 
             'buildingclasstypeid', 
             'buildingqualitytypeid', 
             'decktypeid', 
             'fips', 
             'propertylandusetypeid', 
             'regionidzip', 
             'storytypeid', 
             'typeconstructiontypeid', 
             'censustractandblock', 
             ]
    dfall[stringcol] = dfall[stringcol].astype('string') 
    dfall.info()
    dfall.to_pickle(data_path.joinpath('dfall_typemod.pkl'))
    dfall_typemod = dfall.copy()
    print('save cache')

load cache


In [11]:
dfall_typemod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5970758 entries, 0 to 5970757
Data columns (total 61 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   parcelid                      string 
 1   airconditioningtypeid         string 
 2   architecturalstyletypeid      string 
 3   basementsqft                  Int64  
 4   bathroomcnt                   float64
 5   bedroomcnt                    Int64  
 6   buildingclasstypeid           string 
 7   buildingqualitytypeid         string 
 8   calculatedbathnbr             float64
 9   decktypeid                    string 
 10  finishedfloor1squarefeet      Int64  
 11  calculatedfinishedsquarefeet  Int64  
 12  finishedsquarefeet12          Int64  
 13  finishedsquarefeet13          Int64  
 14  finishedsquarefeet15          Int64  
 15  finishedsquarefeet50          Int64  
 16  finishedsquarefeet6           Int64  
 17  fips                          string 
 18  fireplacecnt          

In [5]:
for colname in dfall_typemod.columns:
    col_tmp = dfall_typemod[colname]
    print(colname)
    print(col_tmp.unique())

parcelid
<StringArray>
[ '10754147',  '10759547',  '10843547',  '10859147',  '10879947',  '10898347',
  '10933547',  '10940747',  '10954547',  '10976347',
 ...
  '14460030',  '14284830',  '14285230',  '14455630',  '11117630', '168176230',
  '14273630', '168040630', '168040830', '168040430']
Length: 2985217, dtype: string
airconditioningtypeid
<StringArray>
[<NA>, '1', '13', '5', '11', '9', '12', '3']
Length: 8, dtype: string
architecturalstyletypeid
<StringArray>
[<NA>, '7', '21', '8', '2', '3', '5', '10', '27']
Length: 9, dtype: string
basementsqft
<IntegerArray>
[<NA>,  216,  555,  224,  782,  651,  516,  600,  732, 1145,
 ...
 1674,  808, 1071,  996,  557,  806,  393,   77,  284,  136]
Length: 752, dtype: Int64
bathroomcnt
[ 0.    2.    4.    3.    1.    2.5   3.5   5.    1.5   4.5   7.5   5.5
  6.    7.   10.    8.    9.   12.   11.    8.5   6.5  13.    9.5  14.
 20.   19.5  15.   10.5    nan 18.   16.    1.75 17.   19.    0.5  12.5
 11.5  14.5  32.   31.  ]
bedroomcnt
<IntegerArra

In [11]:
if (not 'dfall_fillna' in locals()) and data_path.joinpath('dfall_fillna.pkl').is_file():
    print('load cache')
    dfall_fillna = pd.read_pickle(data_path.joinpath('dfall_fillna.pkl')) 
elif (not 'dfall_fillna' in locals()) or (not data_path.joinpath('dfall_fillna.pkl').is_file()):
    fillnadict = {
        'string': {'fillvale': 'NA', 'new_type': 'string'},
        'int64': {'fillvale': -1, 'new_type': 'int64'},
        'Int64': {'fillvale': -1, 'new_type': 'int64'},
        'float64': {'fillvale': 0, 'new_type': 'float64'},
    }

    excludecol = ['logerror']

    def fillnabytypes(df, fillnadict):
        df_new = df.copy()
        for key, value in fillnadict.items():
            coltypeidx = []
            coltypenms = []
            for idx, value in enumerate(df_new.dtypes.items()):
                if str(value[1]) == key:
                    coltypeidx.append(idx)
                    coltypenms.append(value[0])
            for coltypenm in coltypenms:
                nact = sum(df[coltypenm].isna())
                print(f'na ct:{nact}')
                if nact > 0 and (not coltypenm in excludecol): 
                    print('fill na')
                    df_new[coltypenm] = df_new[coltypenm].fillna(fillnadict[key]['fillvale'])
                    print(f'na ct:{sum(df_new[coltypenm].isna())}')
                df_new[coltypenm] = df_new[coltypenm].astype(fillnadict[key]['new_type'])
                print(f'change colname : {coltypenm} type: {key}, fillna: {fillnadict[key]}')
        return df_new
    
    dfall_fillna = fillnabytypes(dfall_typemod, fillnadict)
    dfall_fillna.to_pickle(data_path.joinpath('dfall_fillna.pkl'))

na ct:0
change colname : parcelid type: string, fillna: {'fillvale': 'NA', 'new_type': 'string'}
na ct:4343790
fill na
na ct:0
change colname : airconditioningtypeid type: string, fillna: {'fillvale': 'NA', 'new_type': 'string'}
na ct:5958634
fill na
na ct:0
change colname : architecturalstyletypeid type: string, fillna: {'fillvale': 'NA', 'new_type': 'string'}
na ct:5945398
fill na
na ct:0
change colname : buildingclasstypeid type: string, fillna: {'fillvale': 'NA', 'new_type': 'string'}
na ct:2090658
fill na
na ct:0
change colname : buildingqualitytypeid type: string, fillna: {'fillvale': 'NA', 'new_type': 'string'}
na ct:5936283
fill na
na ct:0
change colname : decktypeid type: string, fillna: {'fillvale': 'NA', 'new_type': 'string'}
na ct:14370
fill na
na ct:0
change colname : fips type: string, fillna: {'fillvale': 'NA', 'new_type': 'string'}
na ct:0
change colname : propertycountylandusecode type: string, fillna: {'fillvale': 'NA', 'new_type': 'string'}
na ct:14370
fill na
na ct:

In [12]:
if (not 'dfall_logerr' in locals()) and data_path.joinpath('dfall_logerr.pkl').is_file():
    print('load cache')
    dfall_logerr = pd.read_pickle(data_path.joinpath('dfall_logerr.pkl')) 
elif (not 'dfall_logerr' in locals()) or (not data_path.joinpath('dfall_logerr.pkl').is_file()):
    dfall_logerr = dfall_fillna[~dfall_fillna['logerror'].isna()]
    dfall_logerr.to_pickle(data_path.joinpath('dfall_logerr.pkl'))

In [13]:
dfall_logerr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167888 entries, 362 to 5970723
Data columns (total 61 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   parcelid                      167888 non-null  string 
 1   airconditioningtypeid         167888 non-null  string 
 2   architecturalstyletypeid      167888 non-null  string 
 3   basementsqft                  167888 non-null  int64  
 4   bathroomcnt                   167888 non-null  float64
 5   bedroomcnt                    167888 non-null  int64  
 6   buildingclasstypeid           167888 non-null  string 
 7   buildingqualitytypeid         167888 non-null  string 
 8   calculatedbathnbr             167888 non-null  float64
 9   decktypeid                    167888 non-null  string 
 10  finishedfloor1squarefeet      167888 non-null  int64  
 11  calculatedfinishedsquarefeet  167888 non-null  int64  
 12  finishedsquarefeet12          167888 non-

In [14]:
for colname in dfall_logerr.columns:
    col_tmp = dfall_logerr[colname]
    print(colname)
    print(col_tmp.unique())

parcelid
<StringArray>
['17073783', '17088994', '17100444', '17102429', '17109604', '17125829',
 '17132911', '17134926', '17139988', '17167359',
 ...
 '11924390', '12139606', '12136406', '13008207', '12006414', '11928618',
 '10813418', '11934023', '11490025', '11864428']
Length: 165210, dtype: string
airconditioningtypeid
<StringArray>
['NA', '1', '5', '13', '11', '9', '3']
Length: 7, dtype: string
architecturalstyletypeid
<StringArray>
['NA', '7', '21', '8', '2', '3', '10']
Length: 7, dtype: string
basementsqft
[  -1  616 1350  676  831 1528  234  485  814  760  196  690  238  802
  168  330 1551 1312  260  184  579  700  585  493 1048  540  100  515
  510  198  771  564  162  608  312  913 1555  671 1210  557  394  314
 1252  396  674  819  912  204  800 1416  384   90  640  380  254  600
 1809  645  786  669  512  273  300  900  200  224 1969   38  604  405
  112 3112  352 1218  588  280  252 3560  126]
bathroomcnt
[ 2.5  1.   2.   1.5  3.   7.   5.   0.   4.   4.5  3.5  5.5  6.  10

In [34]:
dfall_fillna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5970758 entries, 0 to 5970757
Data columns (total 61 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   parcelid                      string 
 1   airconditioningtypeid         string 
 2   architecturalstyletypeid      string 
 3   basementsqft                  UInt64 
 4   bathroomcnt                   float64
 5   bedroomcnt                    int64  
 6   buildingclasstypeid           string 
 7   buildingqualitytypeid         string 
 8   calculatedbathnbr             float64
 9   decktypeid                    string 
 10  finishedfloor1squarefeet      int64  
 11  calculatedfinishedsquarefeet  int64  
 12  finishedsquarefeet12          int64  
 13  finishedsquarefeet13          int64  
 14  finishedsquarefeet15          int64  
 15  finishedsquarefeet50          int64  
 16  finishedsquarefeet6           int64  
 17  fips                          string 
 18  fireplacecnt          